In [1]:
import torch
import torch_geometric

/home/icb/vnicolas.goedert/miniconda3/envs/RFAM/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rfam_dir = "../rfam/data/raw/processed/release-14.8"
rfams = ["RF00001","RF00005","RF00169","RF00050", "RF00020"]

from RNARepLearn.datasets import CombinedRfamDataset, SingleRfamDataset
#dataset = CombinedRfamDataset(rfam_dir, rfams, "Under300", 15, 300)
#dataset = CombinedRfamDataset(rfam_dir, ["RF00001","RF00005"], "RF00001_RF00005")
#dataset = SingleRfamDataset(rfam_dir,"RF00001")
dataset = CombinedRfamDataset(rfam_dir, rfams, "Under1000_RF00001_RF00005_RF00169_RF00050_RF00020", 1000)

from RNARepLearn.utils import train_val_test_loaders

train_loader, val_loader, test_loader = train_val_test_loaders(dataset, 0.8, 0.01, 0.19)

Processing...


RF00001


Processing...
Done!


RF00005


Processing...
Done!


...

Processing...
Done!


RF00020


Processing...
Done!


Processing complete
19085
238
4534


Done!


In [7]:
import numpy as np
import os
for rfam in rfams:
    #print(np.load(os.path.join(rfam_dir, rfam, rfam+".npy"),allow_pickle=True))

{'id': 'DS562887.1/11698675-11698792', 'seq': 'AAAAATAGCAAGAAACCCTGAACGCGCCCGATCTCGTCTGATCTCGGAAGCTAAGCAGGGTCCGGCCTGGTTAGTACTTGGATGGGAGACCGCCTGGGAATATCGGGTGCTGTAGGCT', 'seq_int': array([0, 0, 0, 0, 0, 3, 0, 2, 1, 0, 0, 2, 0, 0, 0, 1, 1, 1, 3, 2, 0, 0,
       1, 2, 1, 2, 1, 1, 1, 2, 0, 3, 1, 3, 1, 2, 3, 1, 3, 2, 0, 3, 1, 3,
       1, 2, 2, 0, 0, 2, 1, 3, 0, 0, 2, 1, 0, 2, 2, 2, 3, 1, 1, 2, 2, 1,
       1, 3, 2, 2, 3, 3, 0, 2, 3, 0, 1, 3, 3, 2, 2, 0, 3, 2, 2, 2, 0, 2,
       0, 1, 1, 2, 1, 1, 3, 2, 2, 2, 0, 0, 3, 0, 3, 1, 2, 2, 2, 3, 2, 1,
       3, 2, 3, 0, 2, 2, 1, 3], dtype=int8), 'len': 118, 'bpp': array([0., 0., 0., ..., 0., 0., 0.]), 'rfam_id': 'RF00001'}
{'id': 'DS233273.1/56842-56915', 'seq': 'GCCCGTGTGGATCAATCGGACCGCGCACTGGACTCACAACCCAGAGGTCGCCGGTTCGAATCCCGCGGCGGGCG', 'seq_int': array([2, 1, 1, 1, 2, 3, 2, 3, 2, 2, 0, 3, 1, 0, 0, 3, 1, 2, 2, 0, 1, 1,
       2, 1, 2, 1, 0, 1, 3, 2, 2, 0, 1, 3, 1, 0, 1, 0, 0, 1, 1, 1, 0, 2,
       0, 2, 2, 3, 1, 2, 1, 1, 2, 2, 3, 3, 1, 2, 0, 0, 3,

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/Masked2501")

In [4]:
from RNARepLearn.modules import LinearEmbedding, RPINetEncoder, AttentionDecoder
layers = []
layers.append(RPINetEncoder(4, 32, 3, 3))
layers.append(AttentionDecoder(32, 4))
model = torch.nn.Sequential(*layers)

In [5]:
from RNARepLearn.train import MaskedTraining, AutoEncoder
training = MaskedTraining(model, 3, 15, writer)
#training = AutoEncoder(model, 3, 15)

In [6]:
training.run(train_loader, val_loader)

[Epoch    1/   3] [Batch    1/ 250] Loss:  2.46e+02 Nucleotide-Loss:  1.38e+00 Edge-Loss:  1.07e+02


KeyboardInterrupt: 

In [14]:
batch = next(iter(test_loader))
true_x = torch.clone(batch.x)
true_edges = torch.clone(batch.edge_weight)

In [15]:
from RNARepLearn.utils import mask_batch, reconstruct_bpp
#nuc_mask, edge_mask = mask_batch(batch,15)

In [17]:
model.double()
nucs, bpp = model(batch)

In [26]:
bpp_n, mask = to_dense_batch(bpp, batch.batch)
bpp_n

torch.Size([32, 123, 3089])

In [27]:
from RNARepLearn.losses import BppReconstructionLoss 
from torch_geometric.utils import to_dense_batch
loss1 = torch.nn.KLDivLoss(reduction="sum")
loss2 = BppReconstructionLoss

In [76]:
loss1(to_dense_batch(bpp, batch.batch)[0].log() , to_dense_batch(torch.tensor(reconstruct_bpp(batch.edge_index, true_edges, (len(bpp),len(bpp)))), batch.batch)[0] )

tensor(25832.2134, dtype=torch.float64, grad_fn=<KlDivBackward0>)

In [77]:
loss2(bpp , torch.tensor(reconstruct_bpp(batch.edge_index, true_edges, (len(bpp),len(bpp)))))

tensor(25832.2134, dtype=torch.float64, grad_fn=<KlDivBackward0>)

In [7]:
bpp

NameError: name 'bpp' is not defined